In [1]:
%run 'table_ui.ipynb'
%run 'viz_ui.ipynb'
%run 'export_ui.ipynb'

Styles()

ResizeTrigger()

In [ ]:
test_tile

TestTile(align_center=True, children=[Card(children=[Html(children=['Download test data File'], tag='h2'), Fle…

In [ ]:
file_tile

FileTile(align_center=True, children=[Card(children=[Html(children=['Select a file'], tag='h2'), Flex(children…

In [ ]:
tb_map_tile

MapTile(align_center=True, children=[Card(children=[Html(children=['Display points'], tag='h2'), Flex(children…

In [ ]:
viz_input_tile

InputTile(align_center=True, children=[Card(children=[Html(children=['set up your inputs'], tag='h2'), Flex(ch…

In [ ]:
export_tile

ExportData(align_center=True, children=[Card(children=[Html(children=['Export data'], tag='h2'), Flex(children…

In [ ]:
export_results

ExportResult(align_center=True, children=[Card(children=[Html(children=['Export results'], tag='h2'), Flex(chi…

In [10]:
[ x for x 
   in [1,2] 
]

[1, 2]